In [1]:

import seaborn as sb
import matplotlib as mp
import os
import glob
import pandas as pd
import tqdm
from DataLoader_pytorch import window_torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch

/Users/hyunsucksong/opt/anaconda3/envs/songforgpu/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_size, shift = 1, 1
train = []
test = []
val = []

using_columns = ['Time[s]', 'TE1', 'E.RPM',
       'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar','E.Id','RPM', 'P_UD']


#x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Tmp', 'E.Pwr', 'E.Tar']
x_columns = [ 'TE1', 'E.RPM', 'E.Vlt', 'E.Iq', 'E.Pwr' ]
print(len(x_columns))


y_columns = ['P_UD']



5


In [3]:
train_datapath = 'csv_final/train'
test_datapath = "csv_final/test"
val_datapath = "csv_final/val"
train_data = os.listdir(train_datapath)
test_data = os.listdir(test_datapath)
val_data = os.listdir(val_datapath)


In [4]:
num = 0
for csv_file in tqdm.tqdm(train_data) : 
    
    df_cut = pd.read_csv('csv_final/train/' + csv_file)[using_columns]
    train.append(df_cut)
    if num > 500 :
        break
    num = num + 1

'''for csv_file in tqdm.tqdm(test_data) : 
    
    df_cut = pd.read_csv('csv_final/test/' + csv_file)[using_columns]
    test.append(df_cut)
    
for csv_file in tqdm.tqdm(val_data) : 
    
    df_cut = pd.read_csv('csv_final/val/' + csv_file)[using_columns]
    val.append(df_cut)'''

100%|█████████████████████████████████████████| 267/267 [00:10<00:00, 25.99it/s]


"for csv_file in tqdm.tqdm(test_data) : \n    \n    df_cut = pd.read_csv('csv_final/test/' + csv_file)[using_columns]\n    test.append(df_cut)\n    \nfor csv_file in tqdm.tqdm(val_data) : \n    \n    df_cut = pd.read_csv('csv_final/val/' + csv_file)[using_columns]\n    val.append(df_cut)"

In [5]:
print(len(train))
print(train)

267
[      Time[s]     TE1   E.RPM  E.Vlt  E.Iq  E.Tmp  E.Pwr   E.Tar  E.Id  RPM  \
0       0.000  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
1       0.001  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
2       0.002  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
3       0.003  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
4       0.004  27.636   986.0   13.0  2.92   74.0    5.0  2928.0   0.0  0.0   
...       ...     ...     ...    ...   ...    ...    ...     ...   ...  ...   
9213    9.213  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9214    9.214  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9215    9.215  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9216    9.216  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   
9217    9.217  27.503  1002.0   13.0  2.90   74.0    5.0  2939.0   0.0  0.0   

       P_UD  
0     1.543  
1     1.545  
2   

In [8]:
train_X, train_Y = window_torch(train, x_size, shift, x_columns, y_columns)
train_dataset = torch.utils.data.TensorDataset(train_X, train_Y)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=len(train_X), shuffle=False) 

''''test_X, test_Y = window_torch(test, x_size, shift, x_columns, y_columns)
test_dataset = torch.utils.data.TensorDataset(test_X, test_Y)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False) 

val_X, val_Y = window_torch(val, x_size, shift, x_columns, y_columns)
val_dataset = torch.utils.data.TensorDataset(val_X, val_Y)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)'''

"'test_X, test_Y = window_torch(test, x_size, shift, x_columns, y_columns)\ntest_dataset = torch.utils.data.TensorDataset(test_X, test_Y)\ntest_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False) \n\nval_X, val_Y = window_torch(val, x_size, shift, x_columns, y_columns)\nval_dataset = torch.utils.data.TensorDataset(val_X, val_Y)\nval_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)"

In [9]:
print(train_X.size())
print(len(list(train_X)))
print(len(train))
print(len(train_X))

torch.Size([2313508, 1, 5])
2313508
267
2313508


In [ ]:
#train_x = train_X.reshape(5036540,5)
#train_y = train_Y.reshape(5036540,5)
#print(train_x)

In [10]:

learning_rate = 0.0000001

num_epochs = 30

num_x_columes = len(x_columns)

#model = nn.Linear(1,1)
model = nn.Linear(num_x_columes,1)
loss_fn = nn.MSELoss()
#loss_fn= nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
print(num_x_columes)

5


In [11]:
def Lrmodel(num_epochs, model, optimizer, train_dl, loss_fn):
    for epoch in range(num_epochs):
        '''pred = model(train_X)
        loss = loss_fn(pred, train_Y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()'''
        
        for xb, yb in train_dl:
            #print("x: ", xb)
            #print("y: ", yb)
            pred = model(xb)
            loss = loss_fn(pred, yb)
            

            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
        
        print('Epochs : [{}]/[{}],  MSE Loss : {} '.format(epoch + 1, num_epochs, loss.item()))

In [ ]:
Lrmodel(num_epochs, model, optimizer, train_loader, loss_fn)

Epochs : [1]/[30],  MSE Loss : 53078.12890625 
Epochs : [2]/[30],  MSE Loss : 23426.16796875 
Epochs : [3]/[30],  MSE Loss : 10328.349609375 
Epochs : [4]/[30],  MSE Loss : 4571.5390625 
Epochs : [5]/[30],  MSE Loss : 2034.394775390625 
Epochs : [6]/[30],  MSE Loss : 916.7232055664062 
Epochs : [7]/[30],  MSE Loss : 426.3446044921875 
Epochs : [8]/[30],  MSE Loss : 209.9185791015625 
Epochs : [9]/[30],  MSE Loss : 115.18047332763672 
Epochs : [10]/[30],  MSE Loss : 73.43489837646484 
Epochs : [11]/[30],  MSE Loss : 55.018043518066406 
Epochs : [12]/[30],  MSE Loss : 46.90379333496094 
Epochs : [13]/[30],  MSE Loss : 43.3292350769043 
Epochs : [14]/[30],  MSE Loss : 41.757164001464844 
Epochs : [15]/[30],  MSE Loss : 41.062461853027344 
Epochs : [16]/[30],  MSE Loss : 40.755306243896484 


In [ ]:

print(list(model.parameters()))
# https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_linear_regression/
# cuda 사용

### 